# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.8 MB/s eta 0:00:00


In [ ]:
!mkdir /content/dataset/
!unzip /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/train3C.zip -d /content/dataset/train3C/
!unzip /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/val3C.zip -d /content/dataset/val3C/

unzip:  cannot find or open /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/train3C.zip, /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/train3C.zip.zip or /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/train3C.zip.ZIP.
unzip:  cannot find or open /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/val3C.zip, /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/val3C.zip.zip or /content/drive/MyDrive/MaiaSpain/CAD/project/dataset/val3C.zip.ZIP.


# Initialization

In [ ]:
%cd /content/drive/MyDrive/MaiaSpain/CAD/project/

[Errno 2] No such file or directory: '/content/drive/MyDrive/MaiaSpain/CAD/project/'
/content


In [ ]:
import os
import numpy as np
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from monai.transforms import (
    Compose, Rand2DElasticd, RandRotate90d, RandFlipd, RandAffined,
    RandCoarseShuffled, EnsureTyped, LoadImaged, Resized, ToTensord, NormalizeIntensityd
)
from monai.data import PersistentDataset
from PIL import Image
import torch
from torch import nn, tensor
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, StepLR
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, cohen_kappa_score
from tqdm import tqdm
from torchvision import models
from shutil import rmtree


# Dataloader

In [ ]:
cls_to_idx = {'bcc':0,'scc':1,'mel':2}
idx_to_cls = {0:'bcc',1:'scc',2:'mel'}

def get_weights(data_path):
  classes = os.listdir(data_path)
  num_samples = {cls: len(os.listdir(os.path.join(data_path, cls))) for cls in classes}
  weights = {cls: (1 / num_samples[cls]) / sum((1 / v for v in num_samples.values())) for cls in classes}
  return weights

def get_data(data_path, cls_weights):
  data = []
  sample_weights = []
  classes = os.listdir(data_path)
  for cls in classes:
      cls_path = os.path.join(data_path, cls)
      for image_name in os.listdir(cls_path):
          image_path = os.path.join(cls_path, image_name)
          data.append({'image':image_path, 'label':cls_to_idx[cls]})
          sample_weights.append(cls_weights[cls])
  return data, sample_weights

In [ ]:
base_transforms = [
    LoadImaged('image',ensure_channel_first=True),
    Resized('image',(224, 224), mode='bilinear'),
    ToTensord(['image','label']),
    NormalizeIntensityd('image')
]

augmentations = [
    #Rand2DElastic('image',prob=0.5, spacing=100, magnitude_range=(3, 12), padding_mode="zeros"),
    RandRotate90d('image',prob=0.5, spatial_axes=[0, 1]),
    RandFlipd('image',prob=0.5, spatial_axis=0),
    RandFlipd('image',prob=0.5, spatial_axis=1),
    #RandAffine('image',prob=0.5,translate_range=(50, 50),rotate_range=(0.75, 0.75),scale_range=(0.25, 0.25),shear_range=(0.25, 0.25),padding_mode="zeros",),
    #RandCoarseShuffle('image',holes=1, spatial_size=50, max_holes=5, max_spatial_size=150, prob=0.5),
]

train_transform = Compose(base_transforms + augmentations)
val_transform = Compose(base_transforms)

# rmtree('/content/cache/',True)

cls_weights = get_weights("/content/drive/MyDrive/CAD/project/dataset/train3C/train")
train_data, train_weights = get_data("/content/drive/MyDrive/CAD/project/dataset/train3C/train", cls_weights)
val_data, val_weights = get_data("/content/drive/MyDrive/CAD/project/dataset/val3C/val", cls_weights)

train_dataset = PersistentDataset(data=train_data, transform=train_transform, cache_dir='/content/cache/train')
train_sampler = WeightedRandomSampler(train_weights, len(train_weights))
train_loader = DataLoader(train_dataset, batch_size=16, sampler=train_sampler, num_workers=os.cpu_count(), pin_memory=True)
val_dataset = PersistentDataset(data=val_data, transform=val_transform, cache_dir='/content/cache/val')
val_loader = DataLoader(val_dataset, batch_size=16, num_workers=os.cpu_count(), pin_memory=True)

# Training and Evaluation

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.ce = nn.CrossEntropyLoss()

    def forward(self, inputs, targets):
        ce_loss = self.ce(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss

def calculate_metrics(labels, preds):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average="macro")
    recall = recall_score(labels, preds, average="macro")
    f1 = f1_score(labels, preds, average="macro")
    kappa = cohen_kappa_score(labels, preds)
    return accuracy, precision, recall, f1, kappa

def train_and_evaluate_model(model, train_loader, val_loader, device, epochs=30, patience=20, min_delta=0.0001):
    model = model.to(device)
    criterion = FocalLoss(alpha=1, gamma=2)
    optimizer = Adam(model.parameters(), lr=1e-3)
    #scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=30*len(train_loader), eta_min=1e-6)
    scheduler=StepLR(optimizer, step_size=25, gamma=0.1)
    best_f1 = 0
    save_dir = "models_newxl/"
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for batch in tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}"):
            images, labels = batch['image'].to(device), batch['label'].to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        scheduler.step()

        print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {train_loss / len(train_loader):.4f}")

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for batch in val_loader:
                images, labels = batch['image'].to(device), batch['label'].to(device)
                outputs = model(images)
                preds = torch.argmax(outputs, dim=1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        accuracy, precision, recall, f1, kappa = calculate_metrics(all_labels, all_preds)
        cm = confusion_matrix(all_labels, all_preds)
        print(f"Validation Metrics - Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}, Kappa: {kappa:.4f}")
        print(f"Confusion Matrix:\n{cm}")

        if f1 > best_f1 + min_delta:
            best_f1 = f1
            model_path = os.path.join(save_dir, f"best_model_epoch_{epoch+1}.pth")
            torch.save(model.state_dict(), model_path)
            print(f"Model saved to {model_path}")
        else:
            patience -= 1
            if patience <= 0:
                print("Early stopping!")
                break

# Experiments

## L

In [ ]:
model = models.efficientnet_v2_l(weights=models.EfficientNet_V2_L_Weights.IMAGENET1K_V1)
num_ftrs = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_ftrs, len(cls_to_idx.keys()))
train_and_evaluate_model(
    model, train_loader, val_loader, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), epochs=100
)

Epoch 1/100:   0%|          | 0/318 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return t

Epoch 1/100, Training Loss: 0.3449


Validation Metrics - Accuracy: 0.5567, Precision: 0.4941, Recall: 0.4612, F1: 0.4351, Kappa: 0.2754
Confusion Matrix:
[[416  49  33]
 [ 75  13   6]
 [305  95 278]]
Model saved to models_new/best_model_epoch_1.pth


Epoch 2/100:   0%|          | 0/318 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return t

Epoch 2/100, Training Loss: 0.2586


...

Validation Metrics - Accuracy: 0.8717, Precision: 0.7882, Recall: 0.8551, F1: 0.8129, Kappa: 0.7761
Confusion Matrix:
[[443  27  28]
 [ 13  76   5]
 [ 55  35 588]]


Epoch 34/100:   0%|          | 0/318 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return 

Epoch 34/100, Training Loss: 0.0091



/usr/local/lib/python3.10/dist-packages/monai/data/dataset.py:374: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(hashfile)
/usr/local/lib/python3.10/dist-

Validation Metrics - Accuracy: 0.8827, Precision: 0.8111, Recall: 0.8311, F1: 0.8204, Kappa: 0.7906
Confusion Matrix:
[[441  21  36]
 [ 15  66  13]
 [ 46  18 614]]
Early stopping!


## XL

In [ ]:
import torch
import timm

checkpoint_path = "/content/drive/MyDrive/CAD/project/models_new/best_model_epoch_11.pth"

model = timm.create_model("tf_efficientnetv2_xl.in21k", pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, len(cls_to_idx.keys()))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load(checkpoint_path, map_location=device)
model.load_state_dict(checkpoint)

model = model.to(device)

train_and_evaluate_model(
    model, train_loader, val_loader, device=device, epochs=100
)


In [ ]:
import timm
model = timm.create_model("tf_efficientnetv2_xl.in21k", pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, len(cls_to_idx.keys()))
train_and_evaluate_model(
    model, train_loader, val_loader, device=torch.device("cuda" if torch.cuda.is_available() else "cpu"), epochs=100
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/943M [00:00<?, ?B/s]

NameError: name 'train_and_evaluate_model' is not defined